In [1]:
import requests

ignore_cpts=["2023-05-06","2023-09-02","2023-09-09","2023-09-20","2023-09-21"]

api_key = ""

def get_historical_stock_data(symbol, qtt, start_date, invested_amount, split=1, split_date = "2030-01-01"):
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=full&apikey={api_key}"

    try:
        response = requests.get(url)
        data = response.json()

        if "Error Message" in data:
            raise Exception(data["Error Message"])

        series_key = "Time Series (Daily)"
        stock_data = []
        for date, values in data[series_key].items():
            if date in ignore_cpts and symbol == "CPTS11.SA":
                continue
            if start_date <= date:
                stock_data.append(parse_prices(qtt, invested_amount, date, values, split, split_date))

        return stock_data

    except Exception as e:
        print(f"An error occurred: {e}")
        return None


def parse_prices(qtt, invested_amount, date, values, split, split_date = "2030-01-01"):
    factor = 1
    if split_date <= date:
        factor = split
    return {
        "date": date,
        "open": float(values["1. open"])* factor,
        "high": float(values["2. high"])* factor,
        "low": float(values["3. low"])* factor,
        "close": float(values["4. close"])* factor,
        "open_profit": float(values["1. open"]) * factor * qtt - invested_amount,
        "high_profit": float(values["2. high"]) * factor * qtt - invested_amount,
        "low_profit": float(values["3. low"]) * factor * qtt - invested_amount,
        "close_profit": float(values["4. close"]) * factor * qtt - invested_amount,
    }

In [2]:
import plotly.graph_objects as go


def plot_candlestick_chart(data, symbol, profit = False):
    sufix = ''
    title="Price"
    if profit:
        sufix = '_profit'
        title="Profit"
    # Extract the necessary data from the stock data
    dates = [item["date"] for item in data]
    opens = [item["open" + sufix] for item in data]
    highs = [item["high" + sufix] for item in data]
    lows = [item["low" + sufix] for item in data]
    closes = [item["close" + sufix] for item in data]

    # Create the candlestick trace
    trace = go.Candlestick(x=dates, open=opens, high=highs, low=lows, close=closes)

    # Create the layout
    layout = go.Layout(title=symbol, yaxis=dict(title=title))

    # Create the figure and add the trace
    fig = go.Figure(data=[trace], layout=layout)

    # Show the figure
    fig.show()

In [3]:
def merge(d1, d2):
    d3 = []
    idxD1, idxD2 = 0, 0
    while idxD1 < len(d1) or idxD2 < len(d2):
        if idxD2 >= len(d2) or (
            idxD1 < len(d1) and d1[idxD1]["date"] > d2[idxD2]["date"]
        ):
            d3.append(d1[idxD1])
            idxD1 += 1
        elif idxD1 >= len(d1) or (
            idxD2 < len(d2) and d1[idxD1]["date"] < d2[idxD2]["date"]
        ):
            d3.append(d2[idxD2])
            idxD2 += 1
        else:
            d3.append(
                {
                    "date": d1[idxD1]["date"],
                    "open_profit": d1[idxD1]["open_profit"] + d2[idxD2]["open_profit"],
                    "high_profit": d1[idxD1]["high_profit"] + d2[idxD2]["high_profit"],
                    "low_profit": d1[idxD1]["low_profit"] + d2[idxD2]["low_profit"],
                    "close_profit": d1[idxD1]["close_profit"] + d2[idxD2]["close_profit"],
                }
            )
            idxD1 += 1
            idxD2 += 1
    return d3

In [4]:
import time
XPCI11 = get_historical_stock_data("XPCI11.SA", 29, invested_amount=2362.05, start_date='2023-05-05')
time.sleep(15)
CPTS11 = get_historical_stock_data("CPTS11.SA", 30, invested_amount=2295.00, start_date='2023-05-05', split=10, split_date='2023-09-20')
time.sleep(15)
KNCR11 = get_historical_stock_data("KNCR11.SA", 25, invested_amount=2378.75, start_date='2023-05-05')
time.sleep(15)
CPTR11 = get_historical_stock_data(symbol="CPTR11.SA", qtt=20, start_date='2023-06-29', invested_amount=1943.20, split=10, split_date='2023-09-20')
time.sleep(15)
HABT11 = get_historical_stock_data(symbol="HABT11.SA", qtt=20, start_date='2023-06-29', invested_amount=1924.40)
time.sleep(15)
ITUB4 = get_historical_stock_data("ITUB4.SA", 38, '2023-06-05', 1026.38)
time.sleep(15)
PETR4 = get_historical_stock_data("PETR4.SA", 71, '2023-06-05', 1951.79)


An error occurred: 'Time Series (Daily)'
An error occurred: 'Time Series (Daily)'


KeyboardInterrupt: 

In [ ]:
plot_candlestick_chart(XPCI11, "XPCI11")
plot_candlestick_chart(CPTS11, "CPTS11")
plot_candlestick_chart(KNCR11, "KNCR11")
plot_candlestick_chart(CPTR11, "CPTR11")
plot_candlestick_chart(HABT11, "HABT11")
plot_candlestick_chart(ITUB4, "ITUB4")
plot_candlestick_chart(PETR4, "PETR4")
plot_candlestick_chart(XPCI11, "XPCI11", True)
plot_candlestick_chart(CPTS11, "CPTS11", True)
plot_candlestick_chart(KNCR11, "KNCR11", True)
plot_candlestick_chart(CPTR11, "CPTR11", True)
plot_candlestick_chart(HABT11, "HABT11", True)
plot_candlestick_chart(ITUB4, "ITUB4", True)
plot_candlestick_chart(PETR4, "PETR4", True)
a = merge(ITUB4, PETR4)
b = merge(a, KNCR11)
c = merge(CPTS11, b)
d = merge(XPCI11, c)
e = merge(CPTR11, d)
tudao = merge(HABT11, e)
plot_candlestick_chart(tudao, "TUDAO", True)